In [1]:
import sys 
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.lines import Line2D
import umap
from scvi.model import CondSCVI,DestVI
import scvi
import diopy

ModuleNotFoundError: No module named 'scvi'

In [ ]:
import time
import os

# 设置工作目录
working_dir = 'E:/doctor/自研/mapping/data/20230814final/h5/st/'
os.chdir(working_dir)

# 获取所有文件列表
datalist = [f for f in os.listdir() if os.path.isfile(f)]

# 循环遍历文件
# for i in range(0, len(datalist)):
for i in range(6, 7):
#     if(datalist[i]!="C2D2_HIP_cellbin.h5"):
#         continue
    print(datalist[i])

    # 读取st数据
    st_path = os.path.join(working_dir, datalist[i])
    st_obj = diopy.input.read_h5(st_path)
    
    # 根据st数据选取sc数据
    if datalist[i] in ["st_HIP_starmap.h5", "C2D2_HIP_cellbin.h5", "HIP_slide_seq.h5", "HIP_test1.h5"]:
        sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_HIP.h5")
        anno = 'annotation'

    elif datalist[i] == "C2D2_CB_cellbin.h5":
        sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_CB.h5")
        anno = 'annotation'

    elif datalist[i] == "C2D2_OB_cellbin.h5":
        sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_OB.h5")
        anno = 'type'

    elif datalist[i] == "st_human_CTX.h5":
        sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_human_CTX.h5")
        anno = 'Subclass'
        

    elif datalist[i] == "st_mouse_CTX.h5":
        sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_mouse_CTX.h5")
        anno = 'Type'
        
    #开始时间
    start_time = time.time()
    sc_adata= sc_obj
    sc_adata.layers["counts"]=sc_adata.X.copy()
    st_adata = st_obj
    st_adata.layers["counts"]=st_adata.X.copy()
    
    st_adata.raw=st_adata
    intersect = np.intersect1d(sc_adata.var_names,st_adata.var_names)
    st_adata=st_adata[:,intersect].copy()
    sc_adata=sc_adata[:,intersect].copy()

    CondSCVI.setup_anndata(sc_adata,layer="counts",labels_key=anno)
    sc_model=CondSCVI(sc_adata,weight_obs=True)
    sc_model.train(max_epochs=250)

    st_adata.layers["counts"]=st_adata.X.copy()
    DestVI.setup_anndata(st_adata,layer="counts")

    st_model=DestVI.from_rna_model(st_adata,sc_model)
    st_model.train(max_epochs=350)
    
    
    print("saving \n")
    #重写一个名称
    name="E:/doctor/自研/mapping/data/20230814final/result/destvi/"+datalist[i]+"_destvi_prediction.csv"
    
    st_adata.obsm["proportions"]=st_model.get_proportions()
    st_adata.obsm["proportions"].to_csv(name)
    
    end_time = time.time()
    time_diff = end_time-start_time
    print(time_diff)
    
    time_name="E:/doctor/自研/mapping/data/20230814final/result/destvi/"+datalist[i]+"_destvi_prediction_time.csv"
    pd.Series([time_diff], name='time_diff').to_csv(time_name, header=True)

    

In [44]:
st_adata

AnnData object with n_obs × n_vars = 7288 × 0
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'imagerow', 'imagecol', 'predict', 'predict_modified', '_indices'
    uns: '_scvi_uuid', '_scvi_manager_uuid'
    layers: 'counts'

In [37]:
i=7
print(datalist[i])

# 读取st数据
st_path = os.path.join(working_dir, datalist[i])
st_obj = diopy.input.read_h5(st_path)

st_mouse_CTX.h5


In [92]:
# 根据st数据选取sc数据
if datalist[i] in ["st_HIP_starmap.h5", "C2D2_HIP_cellbin.h5", "HIP_slide_seq.h5", "HIP_test1.h5"]:
    sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_HIP.h5")
    anno = 'annotation'

elif datalist[i] == "C2D2_CB_cellbin.h5":
    sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_CB.h5")
    anno = 'annotation'

elif datalist[i] == "C2D2_OB_cellbin.h5":
    sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_OB.h5")
    anno = 'type'

elif datalist[i] == "st_CTX_human.h5":
    sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_human_CTX.h5")
    anno = 'Subclass'
    sc_obj.var_names = sc_adata.var_names.str.upper()


elif datalist[i] == "st_CTX_mouse.h5":
    sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_mouse_CTX.h5")
    anno = 'Type'
    
#开始时间
start_time = time.time()
sc_adata= sc_obj
sc_adata.layers["counts"]=sc_adata.X.copy()
st_adata = st_obj
st_adata.layers["counts"]=st_adata.X.copy()
st_adata.raw=st_adata
# intersect = np.intersect1d(sc_adata.var_names,st_adata.var_names)
# st_adata=st_adata[:,intersect].copy()
# sc_adata=sc_adata[:,intersect].copy()

In [93]:
sc_adata

AnnData object with n_obs × n_vars = 17336 × 36517
    obs: 'Subclass', 'nFeature', 'nCount'
    layers: 'counts'

In [95]:
st_adata

AnnData object with n_obs × n_vars = 986 × 3999
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'imagerow', 'imagecol', 'predict', 'predict_modified'
    layers: 'counts'

In [66]:
st_adata.var_names

Index(['Acta2', 'Adam2', 'Adamts2', 'Adamts4', 'Adra1b', 'Alk', 'Ankfn1',
       'Ano4', 'Aqp4', 'Asic4',
       ...
       'Unc13c', 'Unc5b', 'Unc5d', 'Ust', 'Vipr2', 'Vtn', 'Vwc2', 'Wipf3',
       'Wnt7b', 'Zfp804b'],
      dtype='object', name='index', length=234)

In [77]:
sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_mouse_CTX.h5")
sc_obj

AnnData object with n_obs × n_vars = 10485 × 30869
    obs: 'batch', 'n_genes', 'percent_mito', 'n_counts', 'leiden', 'Doublet', 'Doublet Score', 'cluster', 'Class_broad', 'sample', 'Age', 'subclass', 'layer', 'Type', 'Subclass', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.6', 'SCT_snn_res.0.7', 'SCT_snn_res.0.8', 'SCT_snn_res.0.9', 'SCT_snn_res.1', 'seurat_clusters', 'nCount_RNA', 'nFeature_RNA'
    obsm: 'X_pca', 'X_umap'

In [76]:
datalist[i]

'st_mouse_CTX.h5'

In [52]:
intersect = np.intersect1d(sc_adata.var_names,st_adata.var_names)
st_adata=st_adata[:,intersect].copy()
sc_adata=sc_adata[:,intersect].copy()

In [53]:
intersect

array([], dtype=object)

In [31]:
CondSCVI.setup_anndata(sc_adata,layer="counts",labels_key=anno)
sc_model=CondSCVI(sc_adata,weight_obs=True)
sc_model.train(max_epochs=250)

st_adata.layers["counts"]=st_adata.X.copy()
DestVI.setup_anndata(st_adata,layer="counts")

st_model=DestVI.from_rna_model(st_adata,sc_model)
st_model.train(max_epochs=350)

D:\anaconda\envs\cell2loc_env\lib\site-packages\scvi_tools-0.19.0-py3.9.egg\scvi\data\fields\_layer_field.py:91: UserWarning: adata.layers[counts] does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 2/250:   0%|▏                                            | 1/250 [00:03<15:27,  3.72s/it, loss=6.65e+04, v_num=1]

D:\anaconda\envs\cell2loc_env\lib\site-packages\pytorch_lightning-1.7.7-py3.9.egg\pytorch_lightning\trainer\trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
D:\anaconda\envs\cell2loc_env\lib\site-packages\pytorch_lightning-1.7.7-py3.9.egg\pytorch_lightning\trainer\trainer.py:1892: PossibleUserWarning: The number of training batches (8) is smaller than the logging interval Trainer(log_every_n_steps=10). Set a lower value for log_every_n_steps if you want to see logs for

Epoch 350/350: 100%|██████████████████████████████████████████| 350/350 [00:32<00:00, 10.87it/s, loss=1.6e+06, v_num=1]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=350` reached.


Epoch 350/350: 100%|██████████████████████████████████████████| 350/350 [00:32<00:00, 10.65it/s, loss=1.6e+06, v_num=1]


In [3]:
i=6
print(datalist[i])

# 读取st数据
st_path = os.path.join(working_dir, datalist[i])
st_obj = diopy.input.read_h5(st_path)

# 根据st数据选取sc数据
if datalist[i] in ["st_HIP_starmap.h5", "C2D2_HIP_cellbin.h5", "HIP_slide_seq.h5", "HIP_test1.h5"]:
    sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_HIP.h5")
    anno = 'annotation'

elif datalist[i] == "C2D2_CB_cellbin.h5":
    sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_CB.h5")
    anno = 'annotation'

elif datalist[i] == "C2D2_OB_cellbin.h5":
    sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_OB.h5")
    anno = 'type'

elif datalist[i] == "st_CTX_human.h5":
    sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_human_CTX.h5")
    anno = 'Subclass'

elif datalist[i] == "st_CTX_mouse.h5":
    sc_obj = diopy.input.read_h5("E:/doctor/自研/mapping/data/20230814final/h5/sc/sc_mouse_CTX.h5")
    anno = 'Type'
#开始时间
start_time = time.time()
sc_adata= sc_obj
sc_adata.layers["counts"]=sc_adata.X.copy()
st_adata = st_obj
st_adata.layers["counts"]=st_adata.X.copy()

st_adata.raw=st_adata
intersect = np.intersect1d(sc_adata.var_names,st_adata.var_names)
st_adata=st_adata[:,intersect].copy()
sc_adata=sc_adata[:,intersect].copy()

CondSCVI.setup_anndata(sc_adata,layer="counts",labels_key=anno)
sc_model=CondSCVI(sc_adata,weight_obs=True)
sc_model.train(max_epochs=250)

st_adata.layers["counts"]=st_adata.X.copy()
DestVI.setup_anndata(st_adata,layer="counts")

st_model=DestVI.from_rna_model(st_adata,sc_model)
st_model.train(max_epochs=350)


print("saving \n")
#重写一个名称
name="E:/doctor/自研/mapping/data/20230814final/result/destvi/"+datalist[i]+"_destvi_prediction.csv"

st_adata.obsm["proportions"]=st_model.get_proportions()
st_adata.obsm["proportions"].to_csv(name)

end_time = time.time()
time_diff = end_time-start_time
print(time_diff)

time_name="E:/doctor/自研/mapping/data/20230814final/result/destvi/"+datalist[i]+"_destvi_prediction_time.csv"
pd.Series([time_diff], name='time_diff').to_csv(time_name, header=True)

    

6

In [ ]:
i=6
print(datalist[i])

# 读取st数据
st_path = os.path.join(working_dir, datalist[i])
st_obj = diopy.input.read_h5(st_path)